# Test Object Results

In [61]:
import os
import pandas as pd
import re
import altair as alt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [25]:
# Parameters
mageck_mle_outputs = "../../results/development/MAGeCK_MLE/"
mageck_rra_outputs = "../../results/development/MAGeCK_RRA/"
bagel_outputs = "../../results/development/BAGEL/"
crisphiermix_outputs = "../../results/development/CRISPhieRmix/"

In [39]:
results_path = [mageck_mle_outputs, mageck_rra_outputs, bagel_outputs, crisphiermix_outputs]

In [33]:



class ToolResult:
    TOOLS_PATH = {
        'MAGeCK_MLE': '{tool_path}{directory}/{directory}.gene_summary.txt',
        'MAGeCK_RRA': '{tool_path}{directory}/{directory}.gene_summary.txt',
        'BAGEL': '{tool_path}{directory}/{directory}_BAGEL_output.bf',
        'CRISPhieRmix': '{tool_path}{directory}/{directory}.txt'
        }
    
    SEPARATORS = {
        'MAGeCK_MLE': '\t',
        'MAGeCK_RRA': '\t',
        'BAGEL': '\t',
        'CRISPhieRmix': ','
        }
    def __init__(self, path):
        self.path = path
            
            
    def get_tool_name(self):
        m = re.match("^\.\.\/\.\.\/\w+\/\w+\/(\w+)\/$", self.path)
        if m:
            return m.group(1)
        else:
            print('Failed to get tool name.')
            
            
    def create_comparisons_dict(self):
        comparisons_dict = {}
        self.tool = self.get_tool_name()
        for directory in os.listdir(str(self.path)):
            table_file = self.TOOLS_PATH[self.tool].format(tool_path=self.path, directory=directory)
            comparisons_dict[directory] = {'file': table_file, 
                                           'table': pd.read_csv(table_file, sep=self.SEPARATORS[self.tool])}
        self.comparisons_dict = comparisons_dict

    def init(self):
        self.create_comparisons_dict()
        

In [48]:
results = {}
for path in results_path:
    res = ToolResult(path=path)
    res.init()
    results[res.tool] = res
    
    print('>>>>>>>>>>>>>>>>>>>>>>')
    print('Tool:', res.tool)
    print(res.path)
    print('>>>>>>>>>>>>>>>>>>>>>>\n')

>>>>>>>>>>>>>>>>>>>>>>
Tool: MAGeCK_MLE
../../results/development/MAGeCK_MLE/
>>>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>>>
Tool: MAGeCK_RRA
../../results/development/MAGeCK_RRA/
>>>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>>>
Tool: BAGEL
../../results/development/BAGEL/
>>>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>>>
Tool: CRISPhieRmix
../../results/development/CRISPhieRmix/
>>>>>>>>>>>>>>>>>>>>>>



In [55]:
mle = results['MAGeCK_MLE']
genes_summary = mle.comparisons_dict['M07e-J25_vs_M07e-J4']['table']
genes_summary

,Gene,sgRNA,M07e-J4|beta,M07e-J4|z,M07e-J4|p-value,M07e-J4|fdr,M07e-J4|wald-p-value,M07e-J4|wald-fdr,M07e-J25|beta,M07e-J25|z,M07e-J25|p-value,M07e-J25|fdr,M07e-J25|wald-p-value,M07e-J25|wald-fdr
0,RNF17,10,0,NaN,0,0,NaN,NaN,0.162210,2.53920,0.053347,0.161390,1.111000e-02,2.324000e-02
1,MECP2,10,0,NaN,0,0,NaN,NaN,0.114350,1.63040,0.114020,0.267640,1.030100e-01,1.598700e-01
2,ZNHIT1,10,0,NaN,0,0,NaN,NaN,0.161600,2.94820,0.053347,0.161390,3.195900e-03,7.754500e-03
3,PCGF1,10,0,NaN,0,0,NaN,NaN,-0.084868,-1.65040,0.680960,0.793900,9.886400e-02,1.554500e-01
4,SP140,10,0,NaN,0,0,NaN,NaN,-0.014746,-0.23130,0.421550,0.593520,8.170800e-01,8.574400e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951,KIAA0182,10,0,NaN,0,0,NaN,NaN,-0.827630,-6.36300,0.004184,0.023392,1.979100e-10,1.361200e-09
952,FUS,10,0,NaN,0,0,NaN,NaN,-0.036186,-0.66366,0.493720,0.666670,5.069100e-01,5.902600e-01
953,UBA7,10,0,NaN,0,0,NaN,NaN,-0.414940,-4.08460,0.268830,0.452460,4.416100e-05,1.719600e-04
954,PHF14,10,0,NaN,0,0,NaN,NaN,0.230340,3.38360,0.013598,0.064356,7.155100e-04,2.085500e-03


In [169]:
def mageck_mle_preparation(mle, baseline, feature, fdr_cutoff, comparisons):

    def columns_to_drop_mageck_mle(table, baseline):
        z = '{}|z'.format(baseline)
        p_value = '{}|p-value'.format(baseline)
        waldp = '{}|wald-p-value'.format(baseline)
        waldfdr = '{}|wald-fdr'.format(baseline)
        cols = [z, p_value, waldp, waldfdr]
        print(cols)
        return cols

    print('\n-MAGeCK MLE')
    tables_to_concatenate = []
    for comparison in comparisons:
        table = mle.comparisons_dict[comparison]['table']
        tables_to_concatenate.append(table)
        
    result = pd.concat(tables_to_concatenate, axis=1)
    result = result.iloc[:,~result.columns.duplicated()]
    
    return result




@interact(baseline=set([condition['baseline'] for condition in conditions]), feature=widgets.Text(value='MYC', placeholder='Feature to show...', description='Feature:'))
def feature_accros_conditions(baseline, feature):
    @interact(fdr_cutoff=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.05))
    def feature_accros_conditions(fdr_cutoff):
        print('Baseline:', baseline)
        print('Feature:', feature)
        print('FDR cut-off', fdr_cutoff)
        treatments = [condition['treatment'] for condition in conditions if condition['baseline'] == baseline]
        comparisons = [condition['treatment']+'_vs_'+baseline for condition in conditions if condition['baseline'] == baseline]
        print('Comparisons:', comparisons)
#         t = [trt+'|'+str(col) for trt in treatments for col in ['wald-p-value', 'wald-fdr']]
        if mle.tool == 'MAGeCK_MLE':
            result = mageck_mle_preparation(mle, baseline, feature, fdr_cutoff, comparisons)
            info = result.loc[(table['Gene'] == feature)]
        return info
            

interactive(children=(Dropdown(description='baseline', options=('M07e-J4', 'M07e-J11', 'M07e-J18'), value='M07…